In [1]:
import numpy as np
import pandas as pd
import os

from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

from scipy.special import rel_entr
from scipy.stats import entropy
import scipy.stats
import math
import warnings 
warnings.filterwarnings("ignore") 

from sklearn.metrics.cluster import normalized_mutual_info_score, adjusted_rand_score

In [2]:
# 使用GPU
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
! nvidia-smi

Tue Jun 28 16:14:23 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.129.06   Driver Version: 470.129.06   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| 28%   36C    P8    22W / 260W |     22MiB / 11019MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:02:00.0 Off |                  N/A |
| 26%   

# MongoDB

In [4]:
from pymongo import MongoClient
import pymongo

In [5]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

try:
    conn = pymongo.MongoClient(mongoURI)
    db = conn.eva
    # db_de = db.patent_de
    # db_us = db.patent_us
    # db_cn = db.patent_cn

    db_rm_cn_2014 = db.rm_patent_cn_2014
    db_rm_us_2014 = db.rm_patent_us_2014
    db_rm_de_2014 = db.rm_patent_de_2014

    
except errors.ConnectionFailure as err:
    print(err)

# 2014

## Company數量

In [6]:
data_assignee = db_rm_de_2014.aggregate([
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}},
    {"$limit":30}
])

ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"],data["count"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])

In [7]:
df_family_de = pd.DataFrame((zip(ls_company, ls_count)), columns = ['company', 'count'])

In [8]:
data_assignee = db_rm_cn_2014.aggregate([
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}},
    {"$limit":30}
])

ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"],data["count"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])
    
df_family_cn = pd.DataFrame((zip(ls_company, ls_count)), columns = ['company', 'count'])

In [9]:
data_assignee = db_rm_us_2014.aggregate([
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}},
    {"$limit":30}
])

ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"],data["count"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])
    
df_family_us = pd.DataFrame((zip(ls_company, ls_count)), columns = ['company', 'count'])

In [10]:
# final_df = pd.concat([df_family_de,df_family_cn,df_family_us],axis=0)

In [11]:
# 取總數前15家公司
df_family_us.groupby("company").sum().reset_index().sort_values("count",ascending=False).head(15)

,company,count
6,Cisco Technology Inc,85
10,Intel Corp,70
21,Qualcomm Inc,59
27,Telefonaktiebolaget LM Ericsson AB,40
25,Samsung Electronics Co Ltd,24
7,Convida Wireless LLC,21
14,LG Electronics Inc,18
18,Nokia Technologies Oy,15
5,Belkin International Inc,12
16,Microsoft Technology Licensing LLC,12


In [12]:
com_df = df_family_us.groupby("company").sum().reset_index().sort_values("count",ascending=False).head(15)
com_df.reset_index(drop=True, inplace=True)

In [13]:
com_df["count_log"] = com_df["count"].apply(lambda x: np.log(x))
com_df

,company,count,count_log
0,Cisco Technology Inc,85,4.442651
1,Intel Corp,70,4.248495
2,Qualcomm Inc,59,4.077537
3,Telefonaktiebolaget LM Ericsson AB,40,3.688879
4,Samsung Electronics Co Ltd,24,3.178054
5,Convida Wireless LLC,21,3.044522
6,LG Electronics Inc,18,2.890372
7,Nokia Technologies Oy,15,2.708050
8,Belkin International Inc,12,2.484907
9,Microsoft Technology Licensing LLC,12,2.484907


公司名稱正規化

In [14]:
com_df["company"] = com_df.company.apply(lambda x: x.replace(" Inc","")\
                                                    .replace(" Co Ltd","")\
                                                    .replace(" Co. Ltd.","")\
                                                    .replace(" Co., Ltd.","")\
                                                    .replace(" Ltd","")\
                                                    .replace(" Corp","")\
                                                    .replace("..","")\
                                                    .replace("International Business Machines","IBM")\
                                                    .replace("Nippon Telegraph and Telephone","Nippon Telegraph & Tel")\
                                                    .replace("Alibaba Group Holding Ltd","Alibaba")\
                                                    .replace("ZTE Intelligent IoT Technology","ZTE")\
                                                    .replace("AT&T Intellectual Property I LP","AT&T")\
                                                    .replace("Microsoft Technology Licensing LLC","Microsoft")\
                                                    .replace("Telefonaktiebolaget LM Ericsson AB","Ericsson")\
                                                    .replace("Cisco Technology","Cisco Systems")\
                                                    .replace("Verizon Patent and Licensing","Verizon Communications")\
                                                    .replace("Tencent Technology Shenzhen","Tencent Holdings")\
                                                    .replace("China Mobile Communications Group","China Mobile")\
                                                    .replace("Alipay Hangzhou Information Technology","Alibaba Group")\
                                                    .replace("Beijing Xiaomi Mobile Software","Xiaomi")\
                                                    .replace("Nokia Technologies Oy","Nokia")\
                                                    .replace("Amazon Technologies","Amazon.com"))

In [15]:
com_df

,company,count,count_log
0,Cisco Systems,85,4.442651
1,Intel,70,4.248495
2,Qualcomm,59,4.077537
3,Ericsson,40,3.688879
4,Samsung Electronics,24,3.178054
5,Convida Wireless LLC,21,3.044522
6,LG Electronics,18,2.890372
7,Nokia,15,2.708050
8,Belkin International,12,2.484907
9,Microsoft,12,2.484907


In [16]:
com_ls = list(com_df.company)

In [17]:
com_ls.remove('Samsung Electronics')

# Data 2014

富比士2000資料

In [18]:
f_data = pd.read_csv("../../Forbes/Forbes_2014.csv")

In [19]:
f_data[f_data["Industry"]=="Semiconductors"]

,Company,Sector,Industry,Continent,Country,Market Value,Sales,Profits,Assets,Rank,...,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24
21,Samsung Electronics,Information Technology,Semiconductors,Asia,South Korea,186.5,208.9,27.2,202.8,22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
77,Intel,Information Technology,Semiconductors,North America,United States,129.2,52.7,9.6,92.4,78,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
153,Qualcomm,Information Technology,Semiconductors,North America,United States,135.5,25.5,6.8,46.3,154,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
189,Taiwan Semiconductor,Information Technology,Semiconductors,Asia,Taiwan,102.0,20.1,6.3,42.4,190,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
421,Texas Instruments,Information Technology,Semiconductors,North America,United States,51.4,12.2,2.1,18.9,421,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
438,SK Hynix,Information Technology,Semiconductors,Asia,South Korea,24.9,12.9,2.6,19.8,439,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
487,Micron Technology,Information Technology,Semiconductors,North America,United States,25.9,11.3,1.8,19.8,488,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
589,ASML Holding,Information Technology,Semiconductors,Europe,Netherlands,41.2,7.0,1.6,16.8,591,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
866,Applied Materials,Information Technology,Semiconductors,North America,United States,25.0,8.1,0.5,12.2,868,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
962,Broadcom,Information Technology,Semiconductors,North America,United States,18.3,8.2,0.4,11.5,964,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Forbes list  
- 同Industry：1
- 同Sector：2
- 有出現在company list：3
- Company list上有但Forbes沒有：4

In [20]:
forb_ls =[]

for i in com_ls:
    if (f_data["Company"]==i).any():
        if (f_data[f_data.Company==i]["Industry"]=="Semiconductors").any():
            forb_ls.append(1)
        elif (f_data[f_data.Company==i]["Sector"]=="Information Technology").any():
            forb_ls.append(2)
        else:
            forb_ls.append(3)
    else:
        forb_ls.append(4)
    

In [21]:
forb_ls

[2, 1, 1, 2, 4, 3, 2, 4, 2, 4, 4, 4, 4, 3]

## Company list competitor

In [22]:
competitor_ls=[4, 1, 2, 2, 2, 2, 1, 4, 1, 4, 2, 2, 2, 3]

## 計算NMI

In [23]:
# (labels_true, labels_pred)
nmi = normalized_mutual_info_score(forb_ls,competitor_ls)
ari = adjusted_rand_score(forb_ls,competitor_ls)

print("nmi: ",nmi)
print("ari: ", ari)

nmi:  0.34572749182909235
ari:  0.06469165659008465


## 公司名稱和類別

In [24]:
competitor_df = pd.DataFrame((zip(com_ls, competitor_ls, forb_ls)), columns = ['company', '2014_competitor', '2014_forb'])

In [25]:
competitor_df

,company,2014_competitor,2014_forb
0,Cisco Systems,4,2
1,Intel,1,1
2,Qualcomm,2,1
3,Ericsson,2,2
4,Convida Wireless LLC,2,4
5,LG Electronics,2,3
6,Nokia,1,2
7,Belkin International,4,4
8,Microsoft,1,2
9,Electronics and Telecommunications Research In...,4,4


In [26]:
competitor_df.to_csv("../competitor_df/cpc_us_2014.csv", index=False)